# Week 4: Constraint satisfaction problems

Lets code a sudoku solver.

Discuss organization around HWs!!!!

In [182]:
assocs = {
    'WA' : ['NT', 'SA'],
    'NT' : ['WA', 'SA', 'Q'],
    'SA' : ['WA', 'NT', 'Q', 'NSW', 'V'],
    'Q'  : ['NT', 'SA', 'NSW'],
    'NSW': ['Q' , 'SA', 'V'],
    'V'  : ['SA', 'NSW'],
    'T'  : [],
}

In [175]:
all_colors = {'R', 'G', 'B'}

In [176]:
def get_valid_assignments(target_var, color_assignments, assocs):
    impossible_colors = set()
    for child_var in assocs.get(target_var, []):
        color_of_child = color_assignments.get(child_var)
        impossible_colors.add(color_of_child)

    return all_colors - impossible_colors

In [177]:
def is_valid_end_assignment(assocs, color_assignments):
    for node, children in assocs.items():
        if node not in color_assignments and len(children) == 0:
            return False

        color_of_node = color_assignments.get(node)
        for child in children:
            color_of_child = color_assignments.get(child)
            if color_of_node is None or color_of_node == color_of_child:
                return False

    return True

In [178]:
assert is_valid_end_assignment(assocs, dict()) == False, \
    'empty assignmenet should be invalid'

assert not is_valid_end_assignment(assocs, {'WA': 'R', 'NA': 'R', 'SA': 'R', 'Q': 'R', 'NSW': 'R'})

assert not is_valid_end_assignment(assocs, {'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R'})

In [179]:
def find_color_assignment(assocs):
    color_assignments = dict()

    def dfs():
        if is_valid_end_assignment(assocs, color_assignments):
            return color_assignments

        for target_var in variables:
            if target_var in color_assignments:
                continue

            valid_assignments = get_valid_assignments(target_var, color_assignments, assocs)
            if len(valid_assignments) == 0:
                return None

            for color in valid_assignments:
                color_assignments[target_var] = color
                assignment = dfs()
                if assignment is not None:
                    return assignment

    return dfs()

In [181]:
assigment = find_color_assignment(assocs)
assigment

{'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R', 'T': 'R'}

In [17]:
# import resource, sys
# resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
# sys.setrecursionlimit(10**6)

In [7]:
empty_sudoku_str = """
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0
"""

empty_sudoku = empty_sudoku_str.split('\n')
empty_sudoku = [row.strip().split(' ') for row in empty_sudoku if row != '']
empty_sudoku = [[int(el) for el in r] for r in empty_sudoku]

In [18]:
some_sudoku = \
    [["8","3",".",".","7",".",".",".","."]
    ,["6",".",".","1","9","5",".",".","."]
    ,[".","9","8",".",".",".",".","6","."]
    ,["8",".",".",".","6",".",".",".","3"]
    ,["4",".",".","8",".","3",".",".","1"]
    ,["7",".",".",".","2",".",".",".","6"]
    ,[".","6",".",".",".",".","2","8","."]
    ,[".",".",".","4","1","9",".",".","5"]
    ,[".",".",".",".","8",".",".","7","9"]]

some_sudoku = [[0 if el == '.' else int(el) for el in row] for row in some_sudoku]
some_sudoku

[[8, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [9]:
def is_valid_sudoku(sudoku):
    all_nums = set(range(1, 10))
    for y in range(9):
        if set(sudoku[y]) != all_nums:
            return False
    for x in range(9):
        if set(row[x] for row in sudoku) != all_nums:
            return False
    for y in range(9):
        for x in range(9):
            if set(
                el
                for row in sudoku[y // 3: y // 3 + 3]
                for el in row[x // 3:x // 3 + 3]
            ) != all_nums:
                return False
            
    return True

def has_empty(sudoku):
    return 0 in set(el for row in sudoku for el in row)

In [10]:
def get_possible_at_position(sudoku, pos):
    x, y = pos
    impossible_row = set(sudoku[y])
    impossible_col = set(row[x] for row in sudoku)
    impossible_sqr = set(
        el
        for row in sudoku[y // 3: y // 3 + 3]
        for el in row[x // 3:x // 3 + 3]
    )
    impossible = impossible_row.union(impossible_col).union(impossible_sqr)

    return set(range(1, 10)) - impossible

In [11]:
from IPython.display import display

In [14]:
def solve(sudoku):
    sudoku = [row[::] for row in sudoku]

    if is_valid_sudoku(sudoku):
        return True, sudoku
    elif not has_empty(sudoku):
        print('dead end')
        return False, sudoku

    possible = {}
    for y in range(9):
        for x in range(9):
            if sudoku[y][x] == 0:
                possible_at_pos = get_possible_at_position(sudoku, (x, y))
                if len(possible_at_pos) == 0:
                    from ipdb import set_trace
                    set_trace()
                    return False, sudoku
                possible[(x, y)] = possible_at_pos

    possible = list(possible.items())
    possible = sorted(possible, key=lambda x: len(x[1]))

    
    for (x, y), elements in possible:
#         if len(elements) > 1:
#             from ipdb import set_trace
#             set_trace()
    
        for element in elements:
            sudoku[y][x] = element
            valid, solved_sudoku = solve(sudoku)
            if valid:
                return valid, solved_sudoku
            else:
                sudoku[y][x] = 0

    return False, sudoku

In [16]:
# solve(some_sudoku)

In [49]:
empty_sudoku

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]